# Fig. 9: Scattering Layer 

In [ ]:
%run header.py

In [ ]:
# Parameters

mu    = 1.0
neps  = 10
eps   = np.linspace(0.1, 1.0, neps)[::-1]
# eps    = 0.01 * 100**np.linspace(0, 1, neps)[::-1]
ndtau = 80
dtmin = 0.01
dtmax = 100.
dtau  = dtmin * (dtmax / dtmin)**np.linspace(0, 1, ndtau)

In [ ]:
intout = np.zeros((neps, ndtau))
intedb = np.zeros((neps, ndtau))

for ieps in range(neps):
    for idtau in range(ndtau):

        # Set up the analytic scattering solution for a slab
        # we take the Planck function to be B_nu=1 for convenience

        Bnu    = 1.
        ntau   = 100
        tau    = np.linspace(-0.5 * dtau[idtau], 0.5 * dtau[idtau], ntau)
        sqeps  = np.sqrt(eps[ieps])
        sqeps3 = np.sqrt(3 * eps[ieps])
        b      = 1. / ((1 - sqeps) * np.exp(-sqeps3 * dtau[idtau]) + (1 + sqeps))
        Jnu = Bnu * (1. - b * (
            np.exp(-sqeps3 * (0.5 * dtau[idtau] - tau)) +
            np.exp(-sqeps3 * (0.5 * dtau[idtau] + tau))
            ))

        # Integrate the formal transfer equation

        intnu = np.zeros(ntau)
        Snu = eps[ieps] * Bnu + (1 - eps[ieps]) * Jnu
        for i in range(1, ntau):
            dtray = (tau[i] - tau[i - 1]) / mu
            expdtray = np.exp(-dtray)
            srcav = 0.5 * (Snu[i] + Snu[i - 1])
            intnu[i] = expdtray * intnu[i - 1] + (1 - expdtray) * srcav
        intout[ieps, idtau] = intnu[-1]

        # Also compute the result using the modified Eddington-Barbier formula

        tauray = (tau[-1] - tau) / mu
        intedb[ieps, idtau] = (1 - np.exp(-dtau[idtau] / mu)) * \
            np.interp(2. / 3., tauray[::-1], Snu[::-1])

#### Do the plot

In [ ]:
f, ax = plt.subplots(figsize=(3.5, 3.5 * 0.65))

for ieps in range(neps):
    l1, = ax.plot(dtau, intout[ieps, :], label=r'$\epsilon_\nu =$' + '{:.1f}'.format(eps[ieps]))
    l2, = ax.plot(dtau, intedb[ieps, :], ':', c=l1.get_color())

ax.set_xlabel(r'$\Delta\tau$')
ax.set_ylabel(r'$I_\nu^{\mathrm{obs}}/B_\nu(T_d)$')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(0.1, 10)
ax.set_ylim(0.01, 1)
ax.legend(fontsize='x-small')

f.subplots_adjust(
    left=0.14, right=0.96,
    bottom=0.17, top=0.95
    )
f.savefig('figures/fig9_scatter_parscan.pdf')